## Finding Postcodes of Australia

If you ever struggle to find postcodes within striking distance of where you are, here is a simple tool everyone can use in order to locate geographically where any postcode is. Note that some of thes reflect Australia Post Boxes (what they call as 'PO Box'), so even they have been captured here as well.

### **1. Import Packages**

The main packages you need for this exercise are *pandas* and *geopy*. If you do find other useful packages to uplift this, please advise.

In [1]:
import pandas as pd
from geopy.distance import geodesic

### **2. Load Postcode dataset**

Included is a dataset with all of the Australian Postcodes with the Locality Name, State, Longitude and Latitude (the latters have been abbreviated as 'long' and 'lat' respectively). Please note that this is an exhaustive list of postcodes in Australia that have existed since 1967, which was also when Postcodes were introduced in Australia. 

In [16]:
df = pd.read_csv('australian_postcodes_latlon.csv')
print(df)

       postcode      locality state        lat        long
0          1001        SYDNEY   NSW -33.869844  151.208285
1          1002        SYDNEY   NSW -33.869844  151.208285
2          1003        SYDNEY   NSW -33.869844  151.208285
3          1004        SYDNEY   NSW -33.869844  151.208285
4          1005        SYDNEY   NSW -33.869844  151.208285
...         ...           ...   ...        ...         ...
18511      6989    MADDINGTON    WA -32.062357  116.006461
18512      6990      GOSNELLS    WA -32.064297  116.008762
18513      6991     KELMSCOTT    WA -32.116402  116.016306
18514      6992      ARMADALE    WA -32.150885  115.942055
18515      6997  KELMSCOTT DC    WA        NaN         NaN

[18516 rows x 5 columns]


### **3. Identify the Latitude and Longitude of the Australian City Centres**

The dataset and the citation of these Latitude and Longitudes is courtesy https://www.matthewproctor.com/australian_postcodes. Please feel free to visit in case you wanted more exposure to the way in which the data is captured.

### **4. Establish function to find Postcodes within any distance of a given city**

Note that the specific business case being tackled involved finding the postcodes within 40 km of Sydney and Melbourne CBDs. You can however tweak this to any city, and if required, tweak this to any geographic location (remember that we are keeping this within Australia).

In [18]:
def postcodes_within_distance(city, max_distance_km, save_to_csv=True):
    """
    Filters all Australian postcodes within a certain distance (in km) of a major city.

    Parameters:
        city (str): The reference city (e.g., "Melbourne", "Sydney").
        max_distance_km (float): The radius in kilometers (e.g., 40, 100, 200).
        save_to_csv (bool): Whether to export the result as a CSV.

    Returns:
        pd.DataFrame: Postcodes within the specified radius.
    """

    # Normalize city input
    city = city.strip().title()

    # Coordinates for major cities
    city_centres = {
        "Sydney": (-33.8688, 151.2093),
        "Melbourne": (-37.8136, 144.9631),
        "Brisbane": (-27.4698, 153.0251),
        "Perth": (-31.9505, 115.8605),
        "Adelaide": (-34.9285, 138.6007),
        "Canberra": (-35.2809, 149.1300),
        # Add more cities as needed
    }

    if city not in city_centres:
        raise ValueError(f"Invalid city '{city}'. Available cities: {', '.join(city_centres.keys())}")

    centre = city_centres[city]

    # Pre-filter using bounding box
    lat_c, lon_c = centre
    deg_buffer = max_distance_km / 111  # Approximate degree per km

    filtered_df = df[
        (df['lat'] >= lat_c - deg_buffer) & (df['lat'] <= lat_c + deg_buffer) &
        (df['long'] >= lon_c - deg_buffer) & (df['long'] <= lon_c + deg_buffer)
    ].copy()

    # Compute distance
    filtered_df['distance_km'] = filtered_df.apply(
        lambda row: geodesic(centre, (row['lat'], row['long'])).km, axis=1
    )

    result = filtered_df[filtered_df['distance_km'] <= max_distance_km].sort_values('distance_km')

    # Export to CSV
    if save_to_csv:
        filename = f"postcodes_within_{int(max_distance_km)}km_{city.lower()}.csv"
        result.to_csv(filename, index=False)
        print(f"✅ Exported {len(result)} rows to '{filename}'")

    return result

### **5. Pick the Australian City of Interest**

Upon printing below, the output is also exported into a CSV format, which you can then use for your analyses.

In [20]:
# Get postcodes within 40km of Sydney
postcodes_within_distance("Sydney", 40)

✅ Exported 924 rows to 'postcodes_within_40km_sydney.csv'


,postcode,locality,state,lat,long,distance_km
0,1001,SYDNEY,NSW,-33.869844,151.208285,0.149099
95,1163,SYDNEY,NSW,-33.869844,151.208285,0.149099
96,1164,SYDNEY,NSW,-33.869844,151.208285,0.149099
97,1165,SYDNEY,NSW,-33.869844,151.208285,0.149099
99,1167,SYDNEY,NSW,-33.869844,151.208285,0.149099
...,...,...,...,...,...,...
472,2083,BAR POINT,NSW,-33.514585,151.163778,39.514439
704,2157,FOREST GLEN,NSW,-33.548333,151.017778,39.732415
474,2083,CHEERO POINT,NSW,-33.510556,151.193333,39.762889
1004,2257,PRETTY BEACH,NSW,-33.528308,151.347617,39.884006


In [22]:
# Get postcodes within 40km of Melbourne
postcodes_within_distance("Melbourne", 40)

✅ Exported 521 rows to 'postcodes_within_40km_melbourne.csv'


,postcode,locality,state,lat,long,distance_km
6984,3000,MELBOURNE,VIC,-37.814245,144.963173,0.071924
10491,8001,MELBOURNE,VIC,-37.814245,144.963173,0.071924
10504,8051,MELBOURNE,VIC,-37.814245,144.963173,0.071924
10505,8066,MELBOURNE,VIC,-37.814245,144.963173,0.071924
10506,8069,MELBOURNE,VIC,-37.814245,144.963173,0.071924
...,...,...,...,...,...,...
7471,3200,FRANKSTON NORTH,VIC,-38.122852,145.160979,38.478379
10432,3977,SKYE,VIC,-38.105126,145.216233,39.267218
7465,3199,FRANKSTON,VIC,-38.145706,145.126374,39.555210
9724,3757,HUMEVALE,VIC,-37.497861,145.180228,39.939485


An additional perk is that you can see the exact distance as well, as you try and validate the distance. Moreover, it does not factor in the actual travel time. For example Cheero Point in NSW is actually 53 km from Sydney CBD when we take this by road. However this tool only factors the straight-line path (i.e. **as the crow flies**).